# Quick Start
Federated Learning (FL) is a learning paradigm for collaboratively learning models from isolated data without directly sharing privacy information, which helps to satisfy the requirements of privacy protection of the public. **FederatedScope**, a comprehensive platform for FL based on a message-oriented framework, aims to provide easy-used and flexible support for developers who want to quick start and custom task-specific FL procedures.

We first provide an end-to-end standalone example to illustrate how to implement a standard FL task with **FederatedScope**.

## Configurations
**FederatedScope** organize the configuration through `yacs.config.cfgNode` and can be found in `federatedscope.configs`. Please refer to the [official documentation](https://federatedscope.io/refs/index) for specific instructions on how to configure `cfg`.

In [ ]:
from federatedscope.core.configs.config import global_cfg

cfg = global_cfg.clone()
print(cfg)

## 1. Prepare datasets
To run an FL task, firstly you should prepare datasets for FL. The DataZoo provided in **FederatedScope** can help to automatically download and preprocess widely-used public datasets from various FL applications, including computer vision, natural language processing, graph learning, recommendation, etc. Developers can conveniently conduct experiments on the provided dataset via specifying `cfg.data.type = DATASET_NAME` in the configuration. 
We also support developers to adopt custom datasets, please refer to [https://federatedscope.io/docs/own-case/]() for more details about the provided datasets in DataZoo, and refer to [02. Start your own case](./02_start_your_own_case.ipynb) for introducing custom datasets in **FederatedScope**.

In [ ]:
from federatedscope.core.auxiliaries.data_builder import get_data

cfg.data.type = 'femnist'
cfg.data.splits = [0.6, 0.2, 0.2]
cfg.data.batch_size = 10
cfg.data.subsample = 0.05
cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.1307], 'std': [0.3081]}]]

data, modified_cfg = get_data(cfg.clone())
cfg.merge_from_other_cfg(modified_cfg)

## 2. Prepare models
You should specify the model architecture that will be federally trained, such as ConvNet or LSTM. FederatedScope includes the ModelZoo to provide the implementation of famous model architectures for various FL applications. Developers can set up `cfg.model.type = MODEL_NAME` to apply a specific model architecture in FL tasks. We allow developers to use custom models via registering without caring about the federated process. You can refer to [02. Start your own case](./02_start_your_own_case.ipynb) for more details about how to customize a model architecture.

In [ ]:
cfg.model.type = 'convnet2'
cfg.model.out_channels = 62

##  3. Start running an FL task (standalone)
Note that FederatedScope provides a unified interface for both standalone mode and distributed mode, and allows users to change via configuring. 

### Task-specific configuration.
Here we demonstrate how to run a standard FL task with FederatedScope as an example, with setting `cfg.data.type = 'FEMNIST'`and `cfg.model.type = 'ConvNet2'` to run vanilla FedAvg for an image classification task. Now configure the Federated Learning process with `cfg`.

In [ ]:
cfg.use_gpu = False
cfg.eval.freq = 10
cfg.eval.metrics = ['acc', 'loss_regular']

cfg.federate.mode = 'standalone'
cfg.federate.local_update_steps = 5
cfg.federate.total_round_num = 20
cfg.federate.sample_client_num = 5
cfg.federate.client_num = 10

cfg.train.optimizer.lr = 0.001
cfg.train.optimizer.weight_decay = 0.0
cfg.grad.grad_clip = 5.0

cfg.criterion.type = 'CrossEntropyLoss'
cfg.trainer.type = 'cvtrainer'
cfg.seed = 123

### Check `cfg` and set up `logger`
We now need to check if the `cfg` has conflicts with default `cfg` and set up `logger` to monitor. The logs will be output in the Cell below.

In [ ]:
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger

setup_seed(cfg.seed)
update_logger(cfg)


### Ready to go, let's get started!

In [ ]:
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls

Fed_runner = FedRunner(data=data,
                       server_class=get_server_cls(cfg),
                       client_class=get_client_cls(cfg),
                       config=cfg.clone())
Fed_runner.run()

Congratulations, you've learned how to run a complete FL process. Next you can modify `cfg` to try out different built-in features in **FederatedScope**, or you can continue to learn [02. Start your own case](./02_start_your_own_case.ipynb).